In [7]:
import numpy as np
import tensorflow as tf
import os
tf.random.set_seed(5)
from numpy.random import default_rng

np.set_printoptions(precision=2)

In [8]:
num_pc = 1000 # number of PC
input_dim = 720 # BVC input size (720 bc RPLidar spits out a 720-point array)
timestep = 32 * 3
max_dist = 12 # not sure what units here
timestep = 32 * 3
tau_w = 10
PI = tf.constant(np.pi) 
rng = default_rng()

In [9]:
class bvcLayer():
    def __init__(self, max_dist, input_dim, angular_tuning_width=90, n_hd=8, sigma_d=.5):
        '''
        Initializes the boundary vector cell (BVC) layer.

        Parameters:
        max_dist: Max distance that the BVCs respond to. Units depend on the context of the environment.
        input_dim: Size of input vector to the BVC layer (e.g., 720 for RPLidar).
        sigma_ang: Standard deviation for the Gaussian function modeling angular tuning of BVCs (in degrees).
        n_hd: Number of head direction cells.
        sigma_d: Standard deviation for the Gaussian function modeling distance tuning of BVCs.
        '''
        
        self.preferred_distances = np.tile(np.arange(0, max_dist, sigma_d/2), n_hd)[np.newaxis, :]  
        # Preferred distances for each BVC; determines how sensitive each BVC is to specific distances.
        
        self.num_distances = self.preferred_distances.size  
        # Total number of BVC tuning points (number of preferred distances).

        self.in_i = np.repeat(np.linspace(0, input_dim, n_hd, endpoint=False, dtype=int), max_dist/(sigma_d/2))[np.newaxis, :]
        # Indices for input vector, aligning BVCs with specific head directions.

        self.preferred_angles = np.linspace(0, 2*np.pi, input_dim)[self.in_i]  
        # Preferred angles for each BVC (in radians).

        self.angular_tuning_width  = tf.constant(np.deg2rad(angular_tuning_width), dtype=tf.float32)  
        # Angular standard deviation for BVC tuning (converted to radians).

        self.bvc_out = None  # Placeholder for BVC output.

        self.sigma_d = tf.constant(sigma_d, dtype=tf.float32)  
        # Distance standard deviation for BVC tuning.

    def compute_bvc_activation(self, distances, angles):
        # Gaussian function for distance tuning
        distance_gaussian = tf.exp(-(distances[self.input_indices] - self.preferred_distances)**2 / (2 * self.distance_tuning_stddev**2)) / tf.sqrt(2 * PI * self.angular_tuning_width**2)
        
        # Gaussian function for angular tuning
        angular_gaussian = tf.exp(-((angles[self.input_indices] - self.preferred_angles)**2) / (2 * self.angular_tuning_width**2)) / tf.sqrt(2 * PI * self.angular_tuning_width**2)
        
        # Return the product of distance and angular Gaussian functions
        return distance_gaussian * angular_gaussian
    
    # How to use (in driver.py): self.pcn([self.boundaries, np.linspace(0, 2*np.pi, 720, False)], self.hdv, self.context, self.mode, np.any(self.collided))
    def __call__(self, distances, angles):
        return tf.reduce_sum(self.compute_bvc_activation(distances, angles), 0)


In [13]:
test = bvcLayer(max_dist, input_dim)

test.preferred_distances.size

384

In [10]:
class PlaceCellLayer():
    def __init__(self, num_pc, input_dim, timestep, max_dist, n_hd):
        '''
        Initializes the Place Cell Layer.

        Parameters:
        num_pc: Number of place cells in the layer.
        input_dim: Dimension of the input vector to the layer (e.g., 720 for RPLidar).
        timestep: Time step for simulation or learning updates.
        max_dist: Maximum distance that the boundary vector cells (BVCs) respond to.
        n_hd: Number of head direction cells.
        '''
        
        # Number of place cells
        self.num_pc = num_pc
        
        # Initialize the Boundary Vector Cell (BVC) layer
        self.bvcLayer = bvcLayer(max_dist, input_dim)
        
        # Number of BVCs
        self.num_bvc = self.bvcLayer.num_distances
        
        # Recurrent weight matrix connecting place cells to one another, with consideration of head directions
        # Shape: (n_hd, num_pc, num_pc)
        self.w_rec_place_to_place = tf.Variable(np.zeros((n_hd, num_pc, num_pc)), dtype=tf.float32)
        
        # Input weight matrix connecting place cells to BVCs
        # Shape: (num_pc, num_bvc)
        self.w_in = tf.Variable(rng.binomial(1, .2, (num_pc, self.num_bvc)), dtype=tf.float32)
        
        # Recurrent weight matrix for interactions involving head direction cells and place cells
        # Shape: (n_hd, num_pc, num_pc)
        self.w_rec_hd_place = tf.zeros(shape=(n_hd, num_pc, num_pc), dtype=tf.float32)
